In [789]:
import time
import random
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd


In [790]:
solar_data = pd.read_csv('/Users/jingsichen/Politecnico Di Torino Studenti Dropbox/Jingsi Chen/Mac/Desktop/AI/solve/pvwatts_hourly.csv')

In [791]:
# upload pvwatts_hourly.csv
G_max_solar = solar_data['AC System Output (W)'][1:169].tolist()
G_max_solar

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 193.056,
 456.187,
 698.037,
 798.739,
 744.47,
 550.861,
 276.852,
 47.432,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 79.149,
 151.658,
 195.548,
 214.597,
 197.529,
 151.481,
 83.781,
 11.64,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 197.027,
 465.658,
 724.393,
 840.61,
 785.156,
 577.757,
 287.2,
 54.188,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 213.6,
 498.359,
 782.73,
 913.997,
 854.124,
 631.132,
 321.292,
 62.236,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 211.266,
 461.919,
 669.045,
 763.252,
 712.175,
 538.016,
 293.096,
 55.485,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 215.201,
 503.584,
 790.216,
 923.281,
 866.243,
 640.185,
 323.377,
 65.209,
 0.0,
 0

In [792]:
def generate_instance():
    operations_info = {}
    # Each tuple in the list corresponds to (num_operations, power_per_operation)
    processes_details = [(32, 100), (28, 125), (26, 150), (25, 175), (24, 200), (23, 225), (22, 250)]
    # processes_details = [(16, 100), (15, 125), (14, 150), (13, 175), (12, 200), (11, 225), (10, 250)]
    for i, (num_operations, power) in enumerate(processes_details):
        # Set every operation's duration to 1 and power to the specified power
        operations = [{'power': power, 'duration': 1} for _ in range(num_operations)]
        # The maximum time for completion is set to the number of operations
        max_time = num_operations
        operations_info[i] = {'operations': operations, 'max_time': max_time}
    
    return operations_info

instance = generate_instance()

In [793]:
# 创建模型
model = Model("Power_Management")

In [794]:
start_hour = 10
Hours = 30
end_hour = start_hour + Hours
I = range(len(instance))  
F = {i: range(len(instance[i]['operations'])) for i in I} 
operations_info = {i: instance[i]['operations'] for i in I}
operation_power = {i: {k: operations_info[i][k]['power'] for k in F[i]} for i in I}
operation_time = {i: {k: operations_info[i][k]['duration'] for k in F[i]} for i in I}
process_max_time = {i: instance[i]['max_time'] for i in I}
E_max = 250  # ESD 最大容量
alpha, beta, theta = 0.9, 0.9, 0.0015  # 充放电和自放电效率
G_max_grid = 1000   # 电网最大功率
GPU_power_options = [100, 125, 150, 175, 200, 225, 250]  # GPU功率选项
# 设置电价和电网回购电价
electricity_price = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.35, 0.45, 0.45, 0.45, 0.45,
                     0.45, 0.45, 0.45, 0.45, 0.45, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.25]*7  # 每小时的电价
sell_back_price = 0.05  # 电网回购电价

In [795]:
contributions = [1/32, 1/28, 1/26, 1/25, 1/24, 1/23, 1/22]
# contributions = [1/16, 1/15, 1/14, 1/13, 1/12, 1/11, 1/10]

In [796]:
# 创建变量
# 太阳能
G_solar = model.addVars(range(start_hour, end_hour), lb=0, name="G_solar")
P_solar = model.addVars(range(start_hour, end_hour), lb=0, name="P_solar")
R_solar = model.addVars(range(start_hour, end_hour), lb=0, name="R_solar")
W_solar = model.addVars(range(start_hour, end_hour), lb=0, name="W_solar")

for t in range(start_hour, end_hour):
    G_solar[t].ub = G_max_solar[t]
    P_solar[t].ub = G_max_solar[t]
    R_solar[t].ub = G_max_solar[t]
    W_solar[t].ub = G_max_solar[t]

# 电网
G_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="G_grid")
P_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="P_grid")
R_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="R_grid")
W_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="W_grid")

# 电池
D_dc = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_dc")
D_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_grid")
ESD = model.addVars(range(start_hour, end_hour+1), lb=0, ub=E_max, name="ESD")

# GPU功率
# y = model.addVars(range(start_hour, end_hour), I, {i: F[i] for i in I}, vtype=GRB.BINARY, name="y") # 操作选择变量 第t时刻第i个进程的第k个操作是否执行
y = model.addVars(range(start_hour, end_hour), I, F[0], vtype=GRB.BINARY, name="y")
gpu_power = model.addVars(range(start_hour, end_hour), lb=0, ub=max(GPU_power_options), name="gpu_power")
power_selection = model.addVars(range(start_hour, end_hour), len(GPU_power_options), vtype=GRB.BINARY, name="power_selection")

# 目标函数：最小化总电力成本
objective =  quicksum((G_grid[t] * electricity_price[t] - W_grid[t] * sell_back_price) for t in range(start_hour, end_hour))
model.setObjective(objective, sense=GRB.MINIMIZE)

In [797]:
model.addConstrs(P_solar[t] + R_solar[t] + W_solar[t] == G_solar[t] for t in range(start_hour, end_hour)) # 太阳能给出去的总功率 = data center + battery + back to grid 
model.addConstrs(P_grid[t] + R_grid[t] == G_grid[t] for t in range(start_hour, end_hour)) # 电网给出去的总功率 = data center + battery
model.addConstrs(W_solar[t] + beta * D_grid[t] == W_grid[t] for t in range(start_hour, end_hour)) # 卖回电网的功率等于太阳能卖回电网的功率加上电池卖回电网的功率
model.addConstrs(R_solar[t]+R_grid[t] <= (E_max - ESD[t]) for t in range(start_hour, end_hour)) # 电池容量限制
model.addConstrs(ESD[t] == (1-theta) * (ESD[t-1] - D_dc[t-1] - D_grid[t-1] + alpha * (R_solar[t-1] + R_grid[t-1])) for t in range(start_hour+1, end_hour+1)) # 电池容量更新
model.addConstrs(D_dc[t] + D_grid[t] <= ESD[t] for t in range(start_hour, end_hour)) # 电池放电功率不得超过电池容量
model.addConstrs(D_dc[t] + D_grid[t] >= 0 for t in range(start_hour, end_hour)) # 电池放电功率不得为负


{10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Update*>,
 25: <gurobi.Constr *Awaiting Model Update*>,
 26: <gurobi.Constr *Awaiting Model Update*>,
 27: <gurobi.Constr *Awaiting Model Update*>,
 28: <gurobi.Constr *Awaiting Model Update*>,
 29: <gurobi.Constr *Awaiting Model Update*>,
 30: <gurobi.Constr *Awaiting Model Update*>,
 31: <gurobi.Constr *Awaiting Mode

In [798]:
# 每个时间槽的job完成量变量
completion_per_slot = model.addVars(range(start_hour, end_hour), name="completion_per_slot", vtype=GRB.CONTINUOUS)

# 每个时间槽中只能执行一个operation
for t in range(start_hour, end_hour):
    model.addConstr(completion_per_slot[t] == quicksum(y[t, i, k] * contributions[i] for i in I for k in F[i]), name=f"CompletionTime_{t}")
    model.addConstr(quicksum(y[t, i, k] for i in I for k in F[i]) == 1, name=f"OneOperationAtTime_{t}")

# 定义完成率变量
model.addConstr(quicksum(completion_per_slot[t] for t in range(start_hour, end_hour)) >= 1, name="CompleteJob")


<gurobi.Constr *Awaiting Model Update*>

In [799]:
model.addConstrs((P_solar[t] + P_grid[t] + beta * D_dc[t] >= quicksum(y[t, i, k] * instance[i]['operations'][k]['power'] for i in I for k in F[i]) for t in range(start_hour, end_hour)), "PowerBalance")
model.addConstrs((P_solar[t] + P_grid[t] + beta * D_dc[t] >= gpu_power[t] for t in range(start_hour, end_hour)), "DataCenterPowerSupplyMatchesGPUPower")
model.addConstrs((quicksum(power_selection[t, j] * GPU_power_options[j] for j in range(len(GPU_power_options))) == gpu_power[t] for t in range(start_hour, end_hour)),"SelectGPUPower")
model.addConstrs((quicksum(power_selection[t, j] for j in range(len(GPU_power_options))) == 1 for t in range(start_hour, end_hour)),"OnePowerOption")

{10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Update*>,
 25: <gurobi.Constr *Awaiting Model Update*>,
 26: <gurobi.Constr *Awaiting Model Update*>,
 27: <gurobi.Constr *Awaiting Model Update*>,
 28: <gurobi.Constr *Awaiting Model Update*>,
 29: <gurobi.Constr *Awaiting Model Update*>,
 30: <gurobi.Constr *Awaiting Model Update*>,
 31: <gurobi.Constr *Awaiting Mode

In [800]:
model.addConstrs(
    (quicksum(y[t, i, k] * operation_power[i][k] for i in I for k in F[i]) <= gpu_power[t] 
     for t in range(start_hour, end_hour)),"GPUPowerLimit")

{10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update*>,
 24: <gurobi.Constr *Awaiting Model Update*>,
 25: <gurobi.Constr *Awaiting Model Update*>,
 26: <gurobi.Constr *Awaiting Model Update*>,
 27: <gurobi.Constr *Awaiting Model Update*>,
 28: <gurobi.Constr *Awaiting Model Update*>,
 29: <gurobi.Constr *Awaiting Model Update*>,
 30: <gurobi.Constr *Awaiting Model Update*>,
 31: <gurobi.Constr *Awaiting Mode

In [801]:
# 初始条件和容量限制
ESD[start_hour].setAttr(GRB.Attr.LB, 0)
ESD[start_hour].setAttr(GRB.Attr.UB, 0)

In [802]:
# 求解模型
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 421 rows, 7321 columns and 23070 nonzeros
Model fingerprint: 0x7433a2cf
Variable types: 391 continuous, 6930 integer (6930 binary)
Coefficient statistics:
  Matrix range     [3e-02, 2e+02]
  Objective range  [5e-02, 5e-01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 137 rows and 6755 columns
Presolve time: 0.01s
Presolved: 284 rows, 566 columns, 1665 nonzeros
Variable types: 176 continuous, 390 integer (360 binary)
Found heuristic solution: objective 360.8077519

Root relaxation: objective 3.133869e+02, 171 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  313.38687    0   10  360.807

     0     0  313.96160    0    8  314.32437  313.96160  0.12%     -    0s
     0     0  313.96160    0   13  314.32437  313.96160  0.12%     -    0s
     0     0  313.96160    0   13  314.32437  313.96160  0.12%     -    0s
     0     0  313.96160    0   13  314.32437  313.96160  0.12%     -    0s
     0     0  313.96160    0   13  314.32437  313.96160  0.12%     -    0s
     0     0  313.96346    0   14  314.32437  313.96346  0.11%     -    0s
     0     0  313.96346    0    8  314.32437  313.96346  0.11%     -    0s
H    0     0                     314.0216345  313.96346  0.02%     -    0s
     0     0          -    0       314.02163  314.02163  0.00%     -    0s

Cutting planes:
  Gomory: 8
  Cover: 6
  Implied bound: 2
  Clique: 3
  MIR: 4
  StrongCG: 1
  GUB cover: 3
  Zero half: 1
  Relax-and-lift: 9

Explored 1 nodes (385 simplex iterations) in 0.14 seconds (0.03 work units)
Thread count was 8 (of 8 available processors)

Solution count 7: 314.022 314.324 314.413 ... 360.808

O

In [803]:
# 输出结果
total_completion = sum(completion_per_slot[t].X for t in range(start_hour, end_hour))
minimum_cost = model.objVal
results = {}
if model.status == GRB.Status.OPTIMAL:
    # results['status'] = "Optimal Solution Found"
    # results['details'] = []
    results = {
    'status': 'Optimal Solution Found',
    'details': [],
    'total_job_completion': f"{total_completion:.2%}",  # Format as a percentage
    'minimum_cost': minimum_cost
}
    for t in range(start_hour, end_hour):
        time_details = {
            'time': t,
            'G_grid': G_grid[t].X,
            'W_grid': W_grid[t].X,
            'P_grid': P_grid[t].X,
            'R_grid': R_grid[t].X,
            'G_solar': G_solar[t].X,
            'W_solar': W_solar[t].X,
            'P_solar': P_solar[t].X,
            'R_solar': R_solar[t].X,
            'D_dc': D_dc[t].X,
            'D_grid': D_grid[t].X,
            'GPU_power': gpu_power[t].X,
            'operations': [],
            'job_completion': completion_per_slot[t].X 
       
        }
        for i in I:
            for k in range(len(operation_power[i])):
                if y[t, i, k].X > 0.5:
                    time_details['operations'].append(f"Process {i} is executed")
        results['details'].append(time_details)
else:
    results['status'] = "No Optimal Solution Found"

In [804]:
results

{'status': 'Optimal Solution Found',
 'details': [{'time': 10,
   'G_grid': 0.0,
   'W_grid': 573.037,
   'P_grid': 0.0,
   'R_grid': 0.0,
   'G_solar': 698.037,
   'W_solar': 573.037,
   'P_solar': 125.0,
   'R_solar': 0.0,
   'D_dc': 0.0,
   'D_grid': 0.0,
   'GPU_power': 125.0,
   'operations': ['Process 1 is executed'],
   'job_completion': 0.03571428571428571},
  {'time': 11,
   'G_grid': 0.0,
   'W_grid': 398.73900000000003,
   'P_grid': 0.0,
   'R_grid': 0.0,
   'G_solar': 798.739,
   'W_solar': 398.73900000000003,
   'P_solar': 150.0,
   'R_solar': 250.0,
   'D_dc': 0.0,
   'D_grid': 0.0,
   'GPU_power': 150.0,
   'operations': ['Process 2 is executed'],
   'job_completion': 0.038461538461538464},
  {'time': 12,
   'G_grid': 0.0,
   'W_grid': 594.1325,
   'P_grid': 0.0,
   'R_grid': 0.0,
   'G_solar': 744.47,
   'W_solar': 594.1325,
   'P_solar': 125.0,
   'R_solar': 25.337499999999977,
   'D_dc': 0.0,
   'D_grid': 0.0,
   'GPU_power': 125.0,
   'operations': ['Process 1 is exe

In [805]:
total_job_completion = results['total_job_completion']
total_job_completion

'100.03%'

In [806]:

energy_dispatch = results['details']
energy_dispatch_df = pd.DataFrame(energy_dispatch)

energy_dispatch_df['total_job_completion'] = f"{total_completion:.2%}"
energy_dispatch_df['minimum_cost'] = f"${minimum_cost:.2f}"

print(energy_dispatch_df.head(20))


    time      G_grid      W_grid      P_grid  R_grid  G_solar     W_solar  \
0     10    0.000000  573.037000    0.000000     0.0  698.037  573.037000   
1     11    0.000000  398.739000    0.000000     0.0  798.739  398.739000   
2     12    0.000000  594.132500    0.000000     0.0  744.470  594.132500   
3     13    0.000000  397.956051    0.000000     0.0  550.861  397.956051   
4     14    0.000000  126.186941    0.000000     0.0  276.852  126.186941   
5     15    0.000000    0.000000    0.000000     0.0   47.432    0.000000   
6     16    0.000000    0.000000    0.000000     0.0    0.000    0.000000   
7     17   28.330983    0.000000   28.330983     0.0    0.000    0.000000   
8     18  100.000000    0.000000  100.000000     0.0    0.000    0.000000   
9     19  100.000000    0.000000  100.000000     0.0    0.000    0.000000   
10    20  100.000000    0.000000  100.000000     0.0    0.000    0.000000   
11    21  100.000000    0.000000  100.000000     0.0    0.000    0.000000   

In [807]:
energy_dispatch_df.to_csv('Optimization_Results.csv', index=False)
